<a href="https://colab.research.google.com/github/VidhanNahata/PahTum/blob/main/PahTum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys
import math

EMPTY = '.'
WHITE = 'W'   # White plays first
BLACK = 'B'

def ask_choice(prompt, valid):
    while True:
        x = input(prompt).strip().lower()
        if x in valid:
            return x
        print(f"Please enter one of: {', '.join(sorted(valid))}")

def ask_board_size(default_size=7, min_size=3, max_size=25):
    while True:
        s = input(f"Enter board size N (press Enter for {default_size}): ").strip()
        if s == "":
            return default_size
        try:
            n = int(s)
            if n < min_size or n > max_size:
                print(f"Please enter an integer between {min_size} and {max_size}.")
                continue
            return n
        except ValueError:
            print("Please enter a valid integer.")

def make_board(n):
    return [[EMPTY for _ in range(n)] for _ in range(n)]

def board_size(board):
    return len(board)

def print_board(board):
    n = board_size(board)
    header = "   " + " ".join(str(c+1) for c in range(n))
    print(header)
    for r in range(n):
        row = f"{r+1:>2} " + " ".join(board[r][c] for c in range(n))
        print(row)
    print()

def in_bounds(board, r, c):
    n = board_size(board)
    return 0 <= r < n and 0 <= c < n

def generate_scoring_table(n):
    """
    Dynamic scoring based on board size:
    - Only runs of length >= 3 score (and <= n).
    - Points increase superlinearly with both run length and board size.
    - Formula: P(L, N) = round((L - 2)^(1 + N/10) * 2)
    Example:
      N=3: only L=3 -> small points
      N=7: resembles classic growth, with bigger rewards for 5,6,7
    """
    scores = {}
    for L in range(1, n+1):
        if L < 3:
            scores[L] = 0
        else:
            val = round(((L - 2) ** (1 + n/10.0)) * 2)
            scores[L] = int(val)
    return scores

def score_line(scores_for_n, run_len):
    return scores_for_n.get(run_len, 0)

def score_board(board, player, scores_for_n):
    """Sum scores of all horizontal and vertical contiguous runs of 'player'."""
    n = board_size(board)
    total = 0
    # Horizontal
    for r in range(n):
        run = 0
        for c in range(n):
            if board[r][c] == player:
                run += 1
            else:
                if run > 0:
                    total += score_line(scores_for_n, run)
                run = 0
        if run > 0:
            total += score_line(scores_for_n, run)
    # Vertical
    for c in range(n):
        run = 0
        for r in range(n):
            if board[r][c] == player:
                run += 1
            else:
                if run > 0:
                    total += score_line(scores_for_n, run)
                run = 0
        if run > 0:
            total += score_line(scores_for_n, run)
    return total

def all_empty_cells(board):
    n = board_size(board)
    for r in range(n):
        for c in range(n):
            if board[r][c] == EMPTY:
                yield (r, c)

def game_over(board):
    n = board_size(board)
    return all(board[r][c] != EMPTY for r in range(n) for c in range(n))

def parse_move(board, s):
    # Accept "r c" with 1-based indices
    parts = s.strip().replace(',', ' ').split()
    if len(parts) != 2:
        return None
    try:
        r = int(parts[0]) - 1
        c = int(parts[1]) - 1
        if in_bounds(board, r, c):
            return (r, c)
    except ValueError:
        pass
    return None

def is_legal(board, r, c):
    return in_bounds(board, r, c) and board[r][c] == EMPTY

def apply_move(board, r, c, player):
    board[r][c] = player

def copy_board(board):
    return [row[:] for row in board]

def immediate_gain(board, r, c, player, scores_for_n):
    """Evaluate move by placing a stone and measuring player's score increase."""
    before = score_board(board, player, scores_for_n)
    tmp = copy_board(board)
    tmp[r][c] = player
    after = score_board(tmp, player, scores_for_n)
    return after - before

def center_weight(board, r, c):
    n = board_size(board)
    center = (n - 1) / 2.0
    return -abs(r - center) - abs(c - center)

def best_ai_move(board, player, opponent, scores_for_n):
    """Greedy 1-ply: maximize immediate score gain; tiebreak by centrality.
       Lightly penalize giving opponent a big immediate reply."""
    best = None
    best_val = -10**9

    for (r, c) in all_empty_cells(board):
        gain = immediate_gain(board, r, c, player, scores_for_n)

        # Light lookahead: opponent's best immediate reply
        tmp = copy_board(board)
        tmp[r][c] = player
        opp_best = 0
        for (rr, cc) in all_empty_cells(tmp):
            opp_gain = immediate_gain(tmp, rr, cc, opponent, scores_for_n)
            if opp_gain > opp_best:
                opp_best = opp_gain

        value = gain - 0.3 * opp_best + 0.05 * center_weight(board, r, c)

        if value > best_val:
            best_val = value
            best = (r, c)

    return best

def print_rules_and_scoring(n, scores_for_n):
    print("\n=== Rules ===")
    print(f"- Board size: {n} x {n}")
    print("- Two players alternate placing stones on empty squares (no holes).")
    print("- White (W) moves first; Black (B) moves second.")
    print("- Only straight lines (horizontal and vertical) are counted; no diagonals.")
    print("- Scoring counts all contiguous runs of your own stones; runs shorter than 3 score 0.")
    print("- The game ends when the board is full. Highest total wins.")

    print("\n=== Scoring Table (for this board) ===")
    print("(Run length : points)")
    for L in range(1, n+1):
        pts = scores_for_n.get(L, 0)
        marker = "" if L >= 3 else " (no score)"
        print(f"{L}: {pts}{marker}")
    print()

def main():
    print("=== Pah-Tum (no holes) — dynamic scoring per board size ===")
    n = ask_board_size(default_size=7)
    board = make_board(n)
    scores_for_n = generate_scoring_table(n)

    # Show rules and points before starting
    print_rules_and_scoring(n, scores_for_n)

    print("Mode: (1) Human vs Computer  (2) Human vs Human")
    mode = ask_choice("Choose 1 or 2: ", {"1", "2"})

    hvsc = (mode == "1")
    human_is_white = True
    if hvsc:
        fw = ask_choice("Do you want to play first as White? (y/n): ", {"y", "n"})
        human_is_white = (fw == "y")

    print("\nBoard coordinates are row col (1..N). '.' = empty.")
    print_board(board)

    turn = 0  # even: White, odd: Black
    while not game_over(board):
        player = WHITE if (turn % 2 == 0) else BLACK
        opponent = BLACK if player == WHITE else WHITE

        if mode == "2":
            print(f"{'White' if player == WHITE else 'Black'} to move.")
            while True:
                mv = input("Enter move (row col): ")
                pos = parse_move(board, mv)
                if not pos:
                    print("Invalid format. Example: 3 5")
                    continue
                r, c = pos
                if not is_legal(board, r, c):
                    print("That square is not available. Try again.")
                    continue
                apply_move(board, r, c, player)
                break
        else:
            is_human = (player == WHITE and human_is_white) or (player == BLACK and not human_is_white)
            if is_human:
                print(f"Your move ({'White' if player == WHITE else 'Black'}).")
                while True:
                    mv = input("Enter move (row col): ")
                    pos = parse_move(board, mv)
                    if not pos:
                        print("Invalid format. Example: 4 4")
                        continue
                    r, c = pos
                    if not is_legal(board, r, c):
                        print("That square is not available. Try again.")
                        continue
                    apply_move(board, r, c, player)
                    break
            else:
                print(f"Computer ({'White' if player == WHITE else 'Black'}) is thinking...")
                move = best_ai_move(board, player, opponent, scores_for_n)
                if move is None:
                    break
                r, c = move
                apply_move(board, r, c, player)
                print(f"Computer plays: {r+1} {c+1}")

        print_board(board)
        turn += 1

    # Final scoring
    w_score = score_board(board, WHITE, scores_for_n)
    b_score = score_board(board, BLACK, scores_for_n)
    print("=== Final Scores ===")
    print(f"White: {w_score}")
    print(f"Black: {b_score}")
    if w_score > b_score:
        print("White wins!")
    elif b_score > w_score:
        print("Black wins!")
    else:
        print("Draw!")

if __name__ == "__main__":
    try:
        main()
    except KeyboardInterrupt:
        print("\nGame interrupted.")
        sys.exit(0)


=== Pah-Tum (no holes) — dynamic scoring per board size ===
Enter board size N (press Enter for 7): 4

=== Rules ===
- Board size: 4 x 4
- Two players alternate placing stones on empty squares (no holes).
- White (W) moves first; Black (B) moves second.
- Only straight lines (horizontal and vertical) are counted; no diagonals.
- Scoring counts all contiguous runs of your own stones; runs shorter than 3 score 0.
- The game ends when the board is full. Highest total wins.

=== Scoring Table (for this board) ===
(Run length : points)
1: 0 (no score)
2: 0 (no score)
3: 2
4: 5

Mode: (1) Human vs Computer  (2) Human vs Human
Choose 1 or 2: 1
Do you want to play first as White? (y/n): y

Board coordinates are row col (1..N). '.' = empty.
   1 2 3 4
 1 . . . .
 2 . . . .
 3 . . . .
 4 . . . .

Your move (White).
Enter move (row col): 2 2
   1 2 3 4
 1 . . . .
 2 . W . .
 3 . . . .
 4 . . . .

Computer (Black) is thinking...
Computer plays: 2 3
   1 2 3 4
 1 . . . .
 2 . W B .
 3 . . . .
 4 . 